In [1]:
# Setup - import library and load data from local file
import pandas as pd
backgrounds = pd.read_csv("data/background.csv")
backgrounds

,biological_education,bio_database_experience,Any biological software tools you'd like to add to the list? How often do you use it?,years_experience,biological_education.1,computer_experience,programming_langs_free_text,What file types do you commonly use for biological data?,years_experience.1
0,highschool,galaxy = every_day;gnomad = never_heard;interm...,,10,doctoral,r_lang = every_month;python = every_day;perl =...,C/C++,GFF;FASTA;BAM;CSV / TSV,15
1,doctoral,galaxy = every_day;gnomad = never_heard;interm...,MaxQuant Every monthCardinal Everz weak,7,informal,r_lang = every_week;python = year_or_less;perl...,,mzml and imzml;FASTA;CSV / TSV,2
2,doctoral,galaxy = every_month;gnomad = never_heard;inte...,BLAST - every weekPUBCHEM - every month,17,informal,r_lang = year_or_less;python = every_month;per...,MongoDBC/C++,GFF;FASTA;VCF;BAM;CSV / TSV;Excel / Google She...,17
3,masters,galaxy = heard_of;gnomad = never_heard;intermi...,Omim - every monthClinVar - every monthENA - e...,11,masters,r_lang = year_or_less;python = every_day;perl ...,,FASTA;VCF;BAM;CSV / TSV;Excel / Google Sheets ...,9
4,doctoral,galaxy = every_week;gnomad = heard_of;intermin...,"Ensembl REST API, BLAST, PyANI, CyVerse (US/UK)",21,doctoral,r_lang = every_day;python = every_day;perl = e...,"bash shell, GNU tools (awk, sed, curl, etc), jq","JSON, RDF;FASTA;VCF;BAM;CSV / TSV;Excel / Goog...",18
5,doctoral,galaxy = heard_of;gnomad = never_heard;intermi...,BEAST - few times a yearPAML - every weekfigtr...,13,informal,r_lang = every_week;python = every_day;perl = ...,,GFF;FASTA;VCF;BAM;CSV / TSV;Excel / Google She...,5
6,doctoral,galaxy = every_day;gnomad = never_heard;interm...,Aequatus and GeneSeqToFamily: Daily,12,masters,r_lang = heard_of;python = every_week;perl = e...,,Newick;GFF;FASTA;VCF;BAM;CSV / TSV;Excel / Goo...,12
7,doctoral,galaxy = every_day;gnomad = year_or_less;inter...,BLAST (few times a year)BWA/BOWTIE/various RNA...,13,masters,r_lang = year_or_less;python = every_day;perl ...,,GFF;FASTA;VCF;BAM;CSV / TSV,18
8,doctoral,galaxy = year_or_less;gnomad = never_heard;int...,"Jalview. Clustal. RAxML, Cytoscape, Chimera, P...",15,doctoral,r_lang = year_or_less;python = every_week;perl...,,Phylip;FASTA;PDB,10
9,doctoral,galaxy = year_or_less;gnomad = never_heard;int...,"Jalview Proviz, often for teaching",11,doctoral,r_lang = year_or_less;python = every_week;perl...,,I have not actively done research in the last ...,7


In [4]:
bioxp = backgrounds.bio_database_experience.str.split(";",expand=True)
bioxp

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,galaxy = every_day,gnomad = never_heard,intermine = never_heard,biomart = heard_of,molgenis = never_heard,knetminer = never_heard,genecards = never_heard,go = heard_of,ensembl = every_month,ncbi = every_week,pdb = heard_of,biogrid = never_heard,interpro = year_or_less,kegg = heard_of,reactome = heard_of
1,galaxy = every_day,gnomad = never_heard,intermine = heard_of,biomart = heard_of,molgenis = never_heard,knetminer = never_heard,genecards = year_or_less,go = every_month,ensembl = year_or_less,ncbi = every_week,pdb = year_or_less,biogrid = heard_of,interpro = heard_of,kegg = year_or_less,reactome = year_or_less
2,galaxy = every_month,gnomad = never_heard,intermine = heard_of,biomart = year_or_less,molgenis = never_heard,knetminer = never_heard,genecards = every_week,go = every_week,ensembl = every_week,ncbi = every_day,pdb = every_month,biogrid = heard_of,interpro = every_week,kegg = year_or_less,reactome = year_or_less
3,galaxy = heard_of,gnomad = never_heard,intermine = never_heard,biomart = heard_of,molgenis = never_heard,knetminer = never_heard,genecards = heard_of,go = heard_of,ensembl = every_day,ncbi = every_month,pdb = heard_of,biogrid = never_heard,interpro = year_or_less,kegg = never_heard,reactome = heard_of
4,galaxy = every_week,gnomad = heard_of,intermine = year_or_less,biomart = year_or_less,molgenis = heard_of,knetminer = year_or_less,genecards = never_heard,go = every_month,ensembl = every_month,ncbi = every_week,pdb = year_or_less,biogrid = never_heard,interpro = year_or_less,kegg = heard_of,reactome = heard_of
5,galaxy = heard_of,gnomad = never_heard,intermine = never_heard,biomart = every_day,molgenis = heard_of,knetminer = never_heard,genecards = heard_of,go = every_week,ensembl = every_day,ncbi = every_day,pdb = never_heard,biogrid = year_or_less,interpro = year_or_less,kegg = year_or_less,reactome = year_or_less
6,galaxy = every_day,gnomad = never_heard,intermine = year_or_less,biomart = every_month,molgenis = heard_of,knetminer = never_heard,genecards = never_heard,go = heard_of,ensembl = every_day,ncbi = every_month,pdb = every_month,biogrid = never_heard,interpro = year_or_less,kegg = year_or_less,reactome = heard_of
7,galaxy = every_day,gnomad = year_or_less,intermine = year_or_less,biomart = year_or_less,molgenis = never_heard,knetminer = heard_of,genecards = year_or_less,go = year_or_less,ensembl = every_month,ncbi = every_month,pdb = year_or_less,biogrid = year_or_less,interpro = year_or_less,kegg = year_or_less,reactome = year_or_less
8,galaxy = year_or_less,gnomad = never_heard,intermine = year_or_less,biomart = year_or_less,molgenis = never_heard,knetminer = never_heard,genecards = year_or_less,go = every_month,ensembl = every_month,ncbi = every_day,pdb = every_week,biogrid = heard_of,interpro = every_week,kegg = year_or_less,reactome = year_or_less
9,galaxy = year_or_less,gnomad = never_heard,intermine = heard_of,biomart = heard_of,molgenis = never_heard,knetminer = never_heard,genecards = year_or_less,go = every_month,ensembl = year_or_less,ncbi = every_month,pdb = year_or_less,biogrid = heard_of,interpro = every_month,kegg = year_or_less,reactome = year_or_less


In [102]:
#"split a multi choice answer, returning a series with a column 
# header and the responses with the header stripped out."
def split_multi_choice( a_series ):
    # variable to store the final response in as we build it
    response = list()
    # Multi-choice answer sets are separated by ; in SelectSurvey
    series_as_df = a_series.str.split(";",expand=True)
    # extract the name of each column and throw it out, 
    # as it's recorded once per row, which is a bit excessive - 
    # we can just keep it as a column header instead
    for i in series_as_df:
        answer = series_as_df[i]
        split_answer = pd.DataFrame(answer.str.split("=",expand=True))
        # TODO: add a series name function input as this method can be used with multiple Qs
        series_name = "bio_software_" + split_answer[0][0] # the 0th value of this is the same for every column
        series_content = split_answer[1]
        returnSeries = pd.Series(data=series_content, name=series_name)
        response.append(returnSeries)
    return pd.DataFrame(response).transpose()

res = split_multi_choice(backgrounds.bio_database_experience)
res


,bio_software_galaxy,bio_software_gnomad,bio_software_intermine,bio_software_biomart,bio_software_molgenis,bio_software_knetminer,bio_software_genecards,bio_software_go,bio_software_ensembl,bio_software_ncbi,bio_software_pdb,bio_software_biogrid,bio_software_interpro,bio_software_kegg,bio_software_reactome
0,every_day,never_heard,never_heard,heard_of,never_heard,never_heard,never_heard,heard_of,every_month,every_week,heard_of,never_heard,year_or_less,heard_of,heard_of
1,every_day,never_heard,heard_of,heard_of,never_heard,never_heard,year_or_less,every_month,year_or_less,every_week,year_or_less,heard_of,heard_of,year_or_less,year_or_less
2,every_month,never_heard,heard_of,year_or_less,never_heard,never_heard,every_week,every_week,every_week,every_day,every_month,heard_of,every_week,year_or_less,year_or_less
3,heard_of,never_heard,never_heard,heard_of,never_heard,never_heard,heard_of,heard_of,every_day,every_month,heard_of,never_heard,year_or_less,never_heard,heard_of
4,every_week,heard_of,year_or_less,year_or_less,heard_of,year_or_less,never_heard,every_month,every_month,every_week,year_or_less,never_heard,year_or_less,heard_of,heard_of
5,heard_of,never_heard,never_heard,every_day,heard_of,never_heard,heard_of,every_week,every_day,every_day,never_heard,year_or_less,year_or_less,year_or_less,year_or_less
6,every_day,never_heard,year_or_less,every_month,heard_of,never_heard,never_heard,heard_of,every_day,every_month,every_month,never_heard,year_or_less,year_or_less,heard_of
7,every_day,year_or_less,year_or_less,year_or_less,never_heard,heard_of,year_or_less,year_or_less,every_month,every_month,year_or_less,year_or_less,year_or_less,year_or_less,year_or_less
8,year_or_less,never_heard,year_or_less,year_or_less,never_heard,never_heard,year_or_less,every_month,every_month,every_day,every_week,heard_of,every_week,year_or_less,year_or_less
9,year_or_less,never_heard,heard_of,heard_of,never_heard,never_heard,year_or_less,every_month,year_or_less,every_month,year_or_less,heard_of,every_month,year_or_less,year_or_less
